# Les Optimisations de mélanges

## Introduction.
Les optimisations de type mélange permettent par exemple de créer la mixture la moins onéreuse, sous contrainte de coût. Il est également possible de maximiser ou minimiser un autre paramètre.
Dans cette étude, je vais utiliser plusieurs solveurs, et faire des optimisations linéaires et non 
linéaires.

Note :  J'ai travaillé sur ce type de problème pendant des semaines, et les résultats et codes ont été validés car sont les mêmes que Lindo et aussi le problème de mélange exposé par Marc Sevaux et Christian Prins dans le livre "Optimisation avec Excel". Le code proposé trouve également strictement la même solution que celle du concepteur de Python Pulp, dans sa fameuse optimisation " The whiskas blending problem"

Généralement, on utilise des pourcentages pour les problèmes de mélange, ce qui complique un peu la rédaction du code avec le solveur.
De plus, on ne produit pas forcément 100% d'alliage, mais cela peut être une valeur unitaire, par exemple, produire 5000  kgs d'un alliage, cependant, cela reste un problème de mélange.

Faire une optimisation de type mélange et obtenir un éventail de variables de décisions dont le total est égal à 100% peut s'appliquer à un grand nombre d'optimisations, pas forcément à des mixtures liquides!

Le principe général est celui-ci : 
On a plusieurs fournisseurs pour chaque métal, qui ont des prix différents, mais leurs métal ont des 
caractéristiques différentes (Ca peut être la resistance, le teneur en x,  etc). Le solveur va nous proposer l'alliage au meilleur prix.

On est pas du tout forcé de maximiser le profit, on peut maximiser ou minimiser un autre paramètre lors du mélange.

Etude globale proposée par <b>Estelle Derrien - Github estellederrien</b>

CREATION EN COURS

L'exemple du fameux problème de mélange linéaire 
"The Pittsburgh steel blending problem" de Lindo, 
résolu par mon application www.solvgraph.com en mode graphique ( Oui, la solution est la même que celle de l'entreprise Lindo...):

<div style="text-align:center">
<img src="img/blend.png">
</div>

# Sommaire

- 1. Minimisation de coût <b>linéaire.</b>
    - Notre problème de base : Problème de Marc Sevaux et Christian Prins
        - Modélisation mathématique
        - Solution avec Python Pulp
    - Notre problème de base : " The Pittsburgh steel problem" de Lindo
        - Modélisation mathématique
        - Solution avec Python Pulp
    - Notre problème de base : "The whiskas can blending problem" de Python Pulp
        - Modélisation mathématique
        - Solution avec Python Pulp

    ***************************************************************************   
- 2. Minimisation de coût <b>non linéaire</b>
    - Notre problème de base
        - Modélisation mathématique
        - Exemple


# Notre problème de base : Problème de Marc Sevaux et Christian Prins

Le problème exposé par Christian Prins et Marc Sevaux dans le livre " 50 optimisations avec Excel " de Eyrolles.

In [4]:

# Une entreprise a reçu une commande de 5 tonnes d'acier pour bateau (CHristian Prins - Marc Sevaux)

# Il doit avoir les caractéristiques suivantes :
# Elément chimique	Pourcentage Minimal	Pourcentage Maximal
# Carbone(C)	        2	                3
# Cuivre(Cu)	        0.4	                0.6
# Manganèse(Mn)	        1.2	                1.65


# Pour fabriquer cet acier, l'entreprise dispose de 7 matières premières de différents fournisseurs :
 
# Matière première	        C%	    Cu%	    Mn%	    Stocks disponibles (kgs)	Coûts(Euros/kilos)
# Alliage de fer 1	        2.5	    0	    1.3	    4000	                    1.20
# Alliage de fer 2	        3	    0	    0.8	    3000	                    1.50
# Alliage de fer 3	        0	    0.3	    0	    6000	                    0.90
# Alliage de cuivre 1       0	    90	    0	    5000	                    1.30
# Alliage de cuivre 2       0	    96	    4	    2000	                    1.45
# Alliage d'Aluminium 1     0	    0.4	    1.2	    3000	                    1.20
# Alliage d'Aluminium 2     0	    0.6		0       2500	                    1  

# Comment minimiser le coût du mélange ?

# La modélisation mathématique.



<div style="text-align:center">
<img src="img/blending-models.png">
</div>

# Résolution avec Python Pulp
Voici mon code qui trouve la même solution que dans le livre de Marc Sevaux et Christian Prins:

In [5]:
import pulp
from pulp import *

# On veut minimiser le cout de 5 tonnes d'alliage #
prob = LpProblem("Production_5_tonnes",LpMinimize)

# On crée nos variables de décisions
# Ca veut dire qu'on doitchoisir quels sont les alliages les plus judicieux à choisir et on leur donne un nom à chacun
# C'est le nombre de kg de l’alliage i utilisés.
# vu que la valeur du nombre de kgs peut être décimal, on spécifie que c'est une variable de décision de type décimale en écrivant Continuous:
F1 = LpVariable("F1", lowBound = 0,cat='Continuous')
F2 = LpVariable("F2", lowBound = 0,cat='Continuous')
F3 = LpVariable("F3", lowBound = 0,cat='Continuous')
CU1 = LpVariable("CU1", lowBound = 0,cat='Continuous')
CU2 = LpVariable("CU2", lowBound = 0,cat='Continuous')
AL1 = LpVariable("AL1", lowBound = 0,cat='Continuous')
AL2 = LpVariable("AL2", lowBound = 0,cat='Continuous')

# Fonction objectif / min (ci * xi) ou ci est exprimé en cout unitaire par kilo 
prob += 1.20 * F1 + 1.50 * F2 + 0.90 * F3 + 1.30 * CU1 + 1.45 * CU2 +  1.20  *  AL1  +  1  *  AL2 , "Cout total"

#! On ne peut pas utiliser plus que le STOCK exprimé en kgs de  matières premières suivantes ( voir la colonne stock du 2 second tableau)
prob += F1 <= 4000; 
prob += F2 <= 3000; 
prob += F3 <= 6000; 
prob += CU1 <= 5000; 
prob += CU2 <= 2000; 
prob += AL1 <= 3000; 
prob += AL2 <= 2500; 


#! -------------------- Les requirements qualité -----------------------!

#! Carbon content; 
# Ici, 100 veut dire qu'on a un besoin en taux carbone mainimal de 2% de 5000 kgs  , ça fait  100 kgs ( voir aussi la doc lindo du fchier 7 de ce rep)
# par contre, les pourcentages dans la partie gauche sont maintenus au format décimal (idem LINDO et l'exemple du fichier  3 aussi )!
prob += 0.025 * F1 + 0.03 * F2 >= 100; 
prob += 0.025 * F1 + 0.03 * F2 <= 150; 

##! Cuivre content; 
prob += 0.003 * F3 + 0.90 * CU1 + 0.96 * CU2 + 0.004 * AL1 + 0.006 * AL2  >=  20; 
prob += 0.003 * F3 + 0.90 * CU1 + 0.96 * CU2 + 0.004 * AL1 + 0.006 * AL2  <=  30; 

##! Manganese content; Ici, 60 veut dire 1,2% de 5000 kgs est égal à 60 kgs, on trouve la même technique chez LINDO et l'exmple 3 de ce rep
prob += 0.013 * F1 + 0.008 * F2 + 0.04 * CU2 + 0.012 * AL1   >=  60; 
prob += 0.013 * F1 + 0.008 * F2 + 0.04 * CU2 + 0.012 * AL1   <=  82.5;  


#! Finish good requirements; 
# On doit spécifier que le total de nos variables de décisions exprimée en kgs doit être égal à 5000 kgs soit 5 tonnes absolument, c'est donc logique, pas de pourcentages ici.
prob += F1 +  F2 +   F3 +  CU1 +  CU2 +   AL1 +  AL2 ==  5000 , "total"

# On affiche notre résultat dans la console :
prob.writeLP("monAlliage.lp")
prob.solve()
print("Status:", LpStatus[prob.status])

for v in prob.variables():
    print(v.name, "=", v.varValue)

print("Cout total des ingrédients = ", value(prob.objective))

Status: Optimal
AL1 = 574.62426
AL2 = 0.0
CU1 = 0.0
CU2 = 27.612723
F1 = 4000.0
F2 = 0.0
F3 = 397.76302
Cout total des ingrédients =  5887.57427835


# Notre problème de base : " The Pittsburgh steel problem" de Lindo

Source :https://www.lindo.com/downloads/LINGO_text/10-Blending_of_Input_Materials.pdf

 Intérêt : On a des contraintes en pourcentages, et on sait désormais comment gérer cela.
 Pulp trouve pareil que Lindo ! Donc, le programme linéaire gère bien les pourcentages.

In [6]:
import pulp
from pulp import *

# On veut minimiser le cout de 2000 pounds d'alliage # 1 tonne signifie 2000 pounds en Angleterre.
prob = LpProblem("Production_1_tonne",LpMinimize)

# ON crée nos variables de décisions
# Ca veut dire qu'on doitchoisir quels sont les alliages les plus judicieux à choisir et on leur donne un nom à chacun
# C'est le nombre de kg de l’alliage i utilisés.
# vu que la valeur du nombre de kgs peut être décimal, on spécifie que c'est une variable de décision de type décimale en écrivant Continuous:
P1 = LpVariable("P1", lowBound = 0,cat='Continuous')
P2 = LpVariable("P2", lowBound = 0,cat='Continuous')
F1 = LpVariable("F1", lowBound = 0,cat='Continuous')
F2 = LpVariable("F2", lowBound = 0,cat='Continuous')
A1 = LpVariable("A1", lowBound = 0,cat='Continuous')
A2 = LpVariable("A2", lowBound = 0,cat='Continuous')
A3 = LpVariable("A3", lowBound = 0,cat='Continuous')
CB = LpVariable("CB", lowBound = 0,cat='Continuous')
S1 = LpVariable("S1", lowBound = 0,cat='Continuous')
S2 = LpVariable("S2", lowBound = 0,cat='Continuous')
S3 = LpVariable("S3", lowBound = 0,cat='Continuous')

# fonction objectif / min (ci * xi) ou xi est exprimé en cout par pounds et ci est exprimé en dollars
prob += 0.03 * P1 + 0.0645 * P2 + 0.065 * F1 + 0.061 * F2 + 0.1 * A1 +  0.13  *  A2  +  0.119  *  A3  +  0.08  *  CB  +  0.021  *  S1  +  0.02  *  S2  +  0.0195 * S3, "Cout total "

#! On ne peut pas utiliser plus que le STOCK de   matières premières suivantes ( voir le tableau du doc lindo)
prob += CB <= 20; 
prob += S1 <= 200; 
prob += S2 <= 200; 
prob += S3 <= 200; 

#! -------------------- Les requirements qualité -----------------------!
#! Carbon content; Ici, 60 veut dire qu'on a besoin de 3% de 2000 pounds , ça fait  60 ( voir le tableau du doc lindo)
prob += 0.04 * P1 + 0.15 * CB + 0.004 * S1 + 0.001 * S2 + 0.001 * S3 >= 60; 
prob += 0.04 * P1 + 0.15 * CB + 0.004 * S1 + 0.001 * S2 + 0.001 * S3 <= 70; 
##! Chrome content; 
prob += 0.1 * P2 + 0.2 * A2 + 0.08 * A3 >=  6; 
prob += 0.1 * P2 + 0.2 * A2 + 0.08 * A3 <=  9; 
##! Manganese content; 
prob += 0.009 * P1 + 0.045 * P2 + 0.6 * A1 + 0.09 * A2 + 0.33 * A3 + 0.009 * S1 + 0.003 * S2 + 0.003 * S3 >= 27; 
prob += 0.009 * P1 + 0.045 * P2 + 0.6 * A1 + 0.09 * A2 + 0.33 * A3 + 0.009 * S1 + 0.003 * S2 + 0.003 * S3 <= 33; 
#! Silicon content; 
prob +=0.0225 * P1 + 0.15 * P2 + 0.45 * F1 + 0.42 * F2 + 0.18 * A1 + 0.3 * A2 + 0.25 * A3 + 0.3 * CB >=  54; 
prob += 0.0225 * P1 + 0.15 * P2 + 0.45 * F1 + 0.42 * F2 + 0.18 * A1 + 0.3 * A2 + 0.25 * A3 + 0.3 * CB <=  60; 


#! Finish good requirements; 
# On doit spécifier que le total de nos variables de décisions exprimée en pounds doit être égal à 2000 POUNDS soit 1 tonne absolument, c'est donc logique, pas de pourcentages ici.
prob += P1 +  P2 +  F1 +   F2 +  A1 +  A2 +   A3 +  CB + S1 + S2 + S3 ==  2000 , "total"


prob.writeLP("monAlliage.lp")
prob.solve()
print("Status:", LpStatus[prob.status])

for v in prob.variables():
    print(v.name, "=", v.varValue)

print("Total Cost of Ingredients  = ", value(prob.objective))


Status: Optimal
A1 = 14.238863
A2 = 0.0
A3 = 0.0
CB = 0.0
F1 = 0.0
F2 = 22.062052
P1 = 1474.2641
P2 = 60.0
S1 = 200.0
S2 = 29.434959
S3 = 200.0
Total Cost of Ingredients  =  59.556293652
